<a href="https://colab.research.google.com/github/rams1987/GemmaFineTuning/blob/main/GemmaFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 26.3 MB/s eta 0:00:00


In [2]:
import bitsandbytes
print(bitsandbytes.__version__)

0.43.1


In [3]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [4]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [5]:
# At present the model is in 32bit. We have to quantize the model so that it can be loaded
# in RAM. So, I've configured to load the model in 4 bit. Quantization technique - nf4 and the new weithts
# should be updated in float16

model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:


tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
!nvidia-smi

Tue Apr 16 05:54:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0              27W /  70W |   2711MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
import torch

print(torch.cuda.is_available())

True


In [8]:
text = "Quote: Imagination is more,"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more, than knowledge.

I am a self-taught artist, born in 1985 in the beautiful city of Porto Alegre, Brazil.

I have always been interested in art, but I never thought I would be able to make a living


In [9]:
os.environ["WANDB_DISABLED"] = "false"

In [10]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [11]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [14]:
# prompt: how to view the above variable 'data' header?

print(data.column_names)


{'train': ['quote', 'author', 'tags', 'input_ids', 'attention_mask']}


In [15]:
data['train']['author']

['Oscar Wilde',
 'Marilyn Monroe',
 'Albert Einstein',
 'Frank Zappa',
 'Marcus Tullius Cicero',
 'Bernard M. Baruch',
 'William W. Purkey',
 'Dr. Seuss',
 'Mae West',
 'Mahatma Gandhi',
 'Robert Frost',
 'J.K. Rowling,',
 'Albert Camus',
 'Mark Twain',
 'C.S. Lewis,',
 'Elbert Hubbard',
 'Oscar Wilde',
 'Oscar Wilde',
 'Mahatma Gandhi',
 'Martin Luther King Jr.,',
 'Friedrich Nietzsche,',
 'Stephen Chbosky,',
 'Oscar Wilde,',
 'Ralph Waldo Emerson',
 'Rob Siltanen',
 'Narcotics Anonymous',
 'Marilyn Monroe',
 'H. Jackson Brown Jr.,',
 'Andre Gide,',
 'Bob Marley',
 'J.R.R. Tolkien,',
 'J.K. Rowling,',
 'Jane Austen,',
 'Marilyn Monroe',
 'Albert Einstein',
 'J.K. Rowling,',
 'Mark Twain',
 'John Green,',
 'Oscar Wilde,',
 'Neil Gaiman,',
 'Maurice Switzer,',
 'William Shakespeare,',
 'Mark Twain',
 'Allen Saunders',
 'Bill Keane',
 'Thomas A. Edison',
 'Friedrich Nietzsche',
 'Elie Wiesel',
 'Eleanor Roosevelt',
 'Mark Twain',
 'Gordon A. Eadie',
 'douglas adams,',
 'Pablo Neruda,',
 

In [16]:
def formatting_func(example):
    text = f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}"
    return [text]

In [17]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# prompt: how to save the trained model in the above cell?

trainer.save_model("outputs/model")


In [18]:
trainer.train()

Step,Training Loss
1,1.679700
2,0.629900
3,1.022800
4,1.031200
5,0.419000
6,1.227900
7,1.091400
8,0.331900
9,0.560700
10,0.502600


TrainOutput(global_step=100, training_loss=0.14157949967309832, metrics={'train_runtime': 69.1249, 'train_samples_per_second': 5.787, 'train_steps_per_second': 1.447, 'total_flos': 54994550906880.0, 'train_loss': 0.14157949967309832, 'epoch': 66.67})

In [19]:
text = "Quote: A woman is like a tea bag;"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: A woman is like a tea bag; you can’t tell how strong she is until you put her in hot water
Author: Eleanor


In [20]:
trainer.save_model("outputs/model")

In [ ]:
# prompt: while saving the above model I got lots of files. which one is the model?

!ls -l outputs/model


In [21]:
# prompt: how to load the saved model again?

model = AutoModelForCausalLM.from_pretrained("outputs/model",
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
# prompt: get the size of the model? in Gb

size = sum(p.numel() for p in model.parameters())
size_in_gb = size * 4 / (1024**3)
print(f"Model size: {size_in_gb:.2f} GB")


Model size: 5.68 GB
